In [2]:
from zxftools_dev.rag import LoadData

In [3]:
document = LoadData("""
王硕
性别：男
年龄：35岁
职业：金融工程师
教育背景：拥有计算机科学硕士学位
个人经历：
王硕自小对计算机和编程充满热情，高中时期便开始自学编程语言。
大学期间，他积极参与各类编程竞赛，多次获得省级以上奖项。
毕业后，他加入了一家知名的科技公司，担任软件工程师，参与了多个大型项目的开发。正在做3D项目
""")

In [ ]:
# 你我他关系

In [ ]:
from zxftools_dev.tknode.tklib.prompt.intention_recognition import intention_recognition2,intention_recognition_example
from qet.llms import get_llm
class ThoughtRoute():
    def __init__(self,engine):

        # self.select_chat_engine = SimpleChatEngine.from_defaults(system_prompt='''你是一个意图判断器,你会对用户的每个输入进行判断,并归类到,分享信息,索要信息. 这两类中
        # for example:
        # 王成宇是谁?  索要信息
        # 桑文是谁?   索要信息
        # ''')


        system_prompt = '''你是一个意图判断器,你会对用户的每个输入进行判断,并归类到,闲聊,深入思考,信息交换. 这三类中'''

        self.select_chat_engine = SimpleChatEngine.from_defaults(system_prompt=system_prompt)

        self.select_doc = {0: 'react', 1: 'caht'}

        self.NodePost = NodePostprocessorsFactory(NodePostprocessorsType.SimilarityPostprocessor)

        self.retriver = IndexMaker().create_index(
            [self.wrap_document('索要信息', 0), self.wrap_document('分享信息', 1)]).as_retriever()

    def chat(self, words):
        self.select_auto(words)

        result = self.thought.chat(words)
        return result.response

    def wrap_document(self, text='索要信息', id=0):
        doc = Document(text=text)
        doc.metadata['prompt_id'] = id
        return doc

    def select_auto1(self,words):
        llm = get_llm()
        def get_prompt_id(resu):
            if resu == []:
                return None
            else:
                return resu[0].metadata['prompt_id']

        result = llm.complete(intention_recognition2.format(input=words, example=intention_recognition_example)).text
        resu = self.retriver.retrieve(result)
        resu = self.NodePost.postprocess_nodes(resu)
        thought = self.select_doc.get(get_prompt_id(resu))
        return thought

    def select_human(self, id):
        thought = self.thought_dict.get(id)
        return thought

    def select_auto(self, words):
        def get_prompt_id(resu):
            if resu == []:
                return None
            else:
                return resu[0].metadata['prompt_id']

        result = self.select_chat_engine.chat(words).response
        resu = self.retriver.retrieve(result)
        resu = self.NodePost.postprocess_nodes(resu)
        prompt = self.select_doc.get(get_prompt_id(resu))
        # prompt or other
        return prompt



    def pop_chat_history(self):
        chat_history = copy.deepcopy(self.thought.agent.chat_history)
        self.thought.agent.chat_history.clear()
        return chat_history

    def get_chat_history(self):
        chat_history = copy.deepcopy(self.thought.agent.chat_history)
        return chat_history

    def clear_history(self):
        self.thought.agent.chat_history.clear()


In [ ]:
class IndexManager():
    def __init__(self,index_path='data/know_index'):
        self.index = IndexMaker.load_index(index_path)
        
    def get_nodes(self):
        return IndexMaker.get_nodes_from_index(self.index)
    
    def insert(self,nodes):
        self.index.insert_nodes(nodes)
        
    def delete(self,id='e721d635-e6ee-4383-8714-c30054e12776'):
        self.index.docstore.delete_document(id)
        
    def retriever(self):
        return self.index.as_retriever()
    

In [8]:
import os

In [9]:
def datacentre(file:str,verbose=False)->str:
    if verbose:
        print(os.listdir(os.environ.get('datacentre')))
    return os.path.join(os.environ.get('datacentre'),file)

In [ ]:
import pandas as pd

def update_dataframe(existing_df, new_row):
    # concatenate the new row to the existing dataframe
    updated_df = pd.concat([existing_df, new_row], ignore_index=True)
    return updated_df


In [247]:
# %%aigen 添加使用示例,和参数说明
import datetime

def get_time(precision=None):
    """
    获取当前日期和时间
    
    :param precision: 精确度，可选值为 "seconds" 或 None，默认为 None
    :return: 格式化后的日期和时间
    
    # 使用示例
    print(get_time())  # 输出格式化后的日期，精确到天
    print(get_time(precision="seconds"))  # 输出格式化后的日期和时间，精确到秒
    """
    current_date = datetime.datetime.now()
    if precision == "seconds":
        formatted_date = current_date.strftime("%Y-%m-%d %H:%M:%S")
    else:
        formatted_date = current_date.strftime("%Y-%m-%d")
    return formatted_date
